# YOLO v3 Finetuning on AWS

This series of notebooks demonstrates how to finetune pretrained YOLO v3 (aka YOLO3) using MXNet on AWS.

**This notebook** walks through using the [SageMaker Hyperparameter Tuning Job](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html) tool to finding optmized hypterparameter and finetune the model.

**Follow-on** the content of the notebooks shows:

* How to use MXNet YOLO3 pretrained model
* How to use Deep SORT with MXNet YOLO3
* How to create Ground-Truth dataset from images the model mis-detected
* How to finetune the model using the created dataset
* Load your finetuned model and Deploy Sagemaker-Endpoint with it using CPU instance.
* Load your finetuned model and Deploy Sagemaker-Endpoint with it using GPU instance.

## Pre-requisites

This notebook is designed to be run in Amazon SageMaker. To run it (and understand what's going on), you'll need:

* Basic familiarity with Python, [MXNet](https://mxnet.apache.org/), [AWS S3](https://docs.aws.amazon.com/s3/index.html), [Amazon SageMaker](https://aws.amazon.com/sagemaker/)
* To create an **S3 bucket** in the same region, and ensure the SageMaker notebook's role has access to this bucket.
* Sufficient [SageMaker quota limits](https://docs.aws.amazon.com/general/latest/gr/aws_service_limits.html#limits_sagemaker) set on your account to run GPU-accelerated spot training jobs.

## Cost and runtime

Depending on your configuration, this demo may consume resources outside of the free tier but should not generally be expensive because we'll be training on a small number of images. You might wish to review the following for your region:

* [Amazon SageMaker pricing](https://aws.amazon.com/sagemaker/pricing/)

The standard `ml.t2.medium` instance should be sufficient to run the notebooks.

We will use GPU-accelerated instance types for training and hyperparameter optimization, and use spot instances where appropriate to optimize these costs.

As noted in the step-by-step guidance, you should take particular care to delete any created SageMaker real-time prediction endpoints when finishing the demo.

# Step 0: Dependencies and configuration

As usual we'll start by loading libraries, defining configuration, and connecting to the AWS SDKs:

In [1]:
%load_ext autoreload
%autoreload 1

# Built-Ins:
import os
import time
import json
from datetime import date, datetime
from glob import glob
from pprint import pprint
from matplotlib import pyplot as plt

# External Dependencies:
import boto3
import imageio
import sagemaker
import numpy as np
from sagemaker.mxnet import MXNet
from botocore.exceptions import ClientError

/Users/dongkyl/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# restore stored variables
%store -r

In [3]:
session = boto3.session.Session()
region = session.region_name
s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)
smclient = session.client('sagemaker')

In [4]:
iam = boto3.client('iam')

In [5]:
print(bucket.name)

sagemaker-ap-northeast-2-929831892372


## Step 1: Recap output.manifest

In last notebook, we made the *output.manifest* that is containing annotation infromation along with image location. And here is the content of the file.

content is dictionary having 2 essential keys, *labels* and *source-ref*. 
- **labels** - contains information of bounding boxes in the value under key *annotations*.  *class_id* is always *0* because we have only one class *person* in the dataset.
- **source-ref** - same value as in *input.manifest* file

For introduction to model training and deployment, see [**Train a Model with Amazon SageMaker**](http://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-training.html)

In [6]:
output_manifest_path = f'annotations/{job_name}/manifests/output/output.manifest'
output_manifest_obj = bucket.Object(output_manifest_path)
dataset = output_manifest_obj.get()['Body'].read().decode('utf-8').split('\n')

In [7]:
pprint(json.loads(dataset[0]))

{'labels': {'annotations': [{'class_id': 0,
                             'height': 230,
                             'left': 90,
                             'top': 45,
                             'width': 182},
                            {'class_id': 0,
                             'height': 231,
                             'left': 150,
                             'top': 0,
                             'width': 174}],
            'image_size': [{'depth': 3, 'height': 320, 'width': 427}]},
 'labels-metadata': {'class-map': {'0': 'Person'},
                     'creation-date': '2020-03-14T12:51:04.692288',
                     'human-annotated': 'yes',
                     'job-name': 'labeling-job/yolo-job-0',
                     'objects': [{'confidence': 0.09}, {'confidence': 0.09}],
                     'type': 'groundtruth/object-detection'},
 'source-ref': 's3://sagemaker-ap-northeast-2-929831892372/yolo-workshop-batch/images/235.jpg'}


## Step 2: Split dataset into Train and Test datasets

Split dataset into train and test datasets is common procedure in Machine Learning(ML).

There are several methods to do that, and we are going to use the simplest one in here. We are going to shuffle entire dataset and split with ratio of 9:1 for train and test respectively.

After split, you will get 2 files, *train.manifest* and *test.manifest* in the path that *output.manifest* is located.

In [8]:
TRAIN_RATIO = 0.9

# shuffle dataset
np.random.shuffle(dataset)

n_samples_total = len(dataset)
train_test_split_index = round(n_samples_total*TRAIN_RATIO)

# split datasets
train_dataset = dataset[:train_test_split_index]
test_dataset = dataset[train_test_split_index:]

n_samples_train = len(train_dataset)
%store n_samples_train
n_samples_test = len(test_dataset)
%store n_samples_test

# store manifests into localhost
with open(f'train.manifest', 'w') as f:
    for line in train_dataset:
        if not line:
            continue
        f.write(str(line))
        f.write("\n")
    
with open(f'test.manifest', 'w') as f:
    for line in test_dataset:
        if not line:
            continue
        f.write(str(line))
        f.write("\n")
        
# store train/test manifests to s3 bucket where output.manifest is located.
manifest_path = output_manifest_path.rsplit('/', 1)[0]
bucket.upload_file('train.manifest', f'{manifest_path}/train.manifest')
print('Training manifest uploaded to:\n' + f's3://{bucket.name}/{manifest_path}/train.manifest')
bucket.upload_file('test.manifest', f'{manifest_path}/test.manifest')
print('Test manifest uploaded to:\n' + f"s3://{bucket.name}/{manifest_path}/test.manifest")

Stored 'n_samples_train' (int)
Stored 'n_samples_test' (int)
Training manifest uploaded to:
s3://sagemaker-ap-northeast-2-929831892372/annotations/yolo-job-0/manifests/output/train.manifest
Test manifest uploaded to:
s3://sagemaker-ap-northeast-2-929831892372/annotations/yolo-job-0/manifests/output/test.manifest


## Step 3: Create Hyperparameter Tuning Job

Now, you are ready to finetune MXNet YOLO model with *train.manifest* and *test.manifest* datasets.

Of course, you create hyperparameter tuning job on AWS Console but there is much easier way to do the same job on sagemaker notebook.

Sagemaker provide *sagemaker.mxnet.MXNet* estimator to train model. With this class you can train or make hyperparameter tuning job for your own model.

First of all, you should define metric for estimator. The estimator's goal is mininize or maximize the metric you gave to it.

In this chapter we are going to use *Loss* as a metric which means the goal of the estimator is going to be minize it as much as it can. 

The estimator container will automatically capture it's *stdout* and find the *Regex* pattern you difined and make it as metric to minize.

In [9]:
metric_definitions = [
    { 'Name': 'TrainLoss', 'Regex': 'Train Loss: (.*?) ;' },
]

If you have execution role for sagemaker just put it to the *role_name* on below cell.

Let's make IAM role on [**AWS Console**]() with *AmazonSagemakerFullAccess* Policy like below screen. 

<img src="Assets/ExecutionRole.png" />

In [10]:
# replace role_name with yours
role_name = 'AmazonSageMaker-ExecutionRole-20200129T183159'
%store role_name

Stored 'role_name' (str)


In [11]:
role = iam.get_role(RoleName=role_name)['Role']['Arn']

Let's make estiamtor. Estimator handles end-to-end Amazon SageMaker training and deployment tasks.

You can run your training job on *Spot Instance* and we are going to do that, because using spot instance is the most cost efficient way to run your job on AWS.

The estimator we are making, uses *4 of ml.p3.8xlarge Spot instances* for training so that 4 Hyperparameter tuning job is able to run cucurrently.

Let me explain some important parameters before run the code,

* entry_point - python script that includes train/finetune logics.
* source_dir - local folder location that `entry_point` is placed.
* frame_work_version - MXNet framework version
* input_mode - 'File' or 'Pipe'. entry_point should be implemented considering input_mode.
* train_use_spot_instances - True if you want to use spot-instance for running training jobs.
* output_path - s3 bucket path that models and checkpoints will be stored.
* hyperparameters - default hyperparameters. most of the values will be overriden by hyperparameter tuning job. (look into *hyperparameter_ranges* variable below cell)

In [12]:
model_output_path = f's3://{BUCKET_NAME}/{MODELS_PREFIX}'
%store model_output_path

Stored 'model_output_path' (str)


In [13]:
estimator = MXNet(
    role=role,
    entry_point='yolo_finetune.py',
    source_dir='src',
    framework_version='1.4.1',
    py_version='py3',
    input_mode='File',
    train_volume_size=n_samples_train,
    train_instance_count=1,
    train_instance_type='ml.p3.8xlarge',
    train_max_run=5*60*60,
    train_use_spot_instances=True,
    train_max_wait=5*60*60,
    metric_definitions=metric_definitions,
    base_job_name='yolo-finetune-0',
    output_path=model_output_path,
    hyperparameters={
        'epochs': 30,
        'num-workers': 4,
        'batch-size': 8,
        'num-gpus': 4,
        'data-shape': 320,
        'lr': 0.000361,
        'momentum': 0.299848,
        'wd': 0.986724,
        'optimizer': 'sgd',
    }
)

## Step 5: Prepare channel

HyperParameter tuning job requires data channel for fetch data from s3.

Estimator on *File* mode, *image_channel* must be provided to the tuner because Sagemaker training container copies all train/test images on creating container instance using *image_channel*.

We are using *File* mode because our dataset is small enough but if you are planning to deal with very large dataset consider *Pipe* mode.

In [14]:
# pass only essential keys
attribute_names = ['source-ref', 'labels']

In [15]:
train_channel = sagemaker.session.s3_input(
    f's3://{BUCKET_NAME}/{manifest_path}/train.manifest',
    distribution='FullyReplicated',
    s3_data_type="S3Prefix",
    attribute_names=attribute_names
)
                                        
test_channel = sagemaker.session.s3_input(
    f's3://{BUCKET_NAME}/{manifest_path}/test.manifest',
    distribution='FullyReplicated',
    s3_data_type='S3Prefix',
    attribute_names=attribute_names
)

image_channel = sagemaker.session.s3_input(
    f's3://{BUCKET_NAME}/{BATCH_NAME}/{IMAGE_PREFIX}',
    s3_data_type="S3Prefix"
)

## Step 6: Finetune Model using Hyperparameter tuning job

[How Hyperparameter Tuning Works](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html) says dailed information about hyperparmeter tuning job.

Simply put it, hyperparameter tuning job test all of the *likely* parameters in the given range, and find best combination of the parameters for the model with given dataset.

In this manner, you should provide ranges of the hyperparameters where the best parameters lie on.

In [16]:
hyperparameter_ranges = {
    'lr': sagemaker.tuner.ContinuousParameter(0.0001, 0.1),
    'momentum': sagemaker.tuner.ContinuousParameter(0.0, 0.99),
    'wd': sagemaker.tuner.ContinuousParameter(0.0, 0.99),
    'optimizer': sagemaker.tuner.CategoricalParameter(['sgd', 'adam', 'rmsprop', 'adadelta'])
}

Put the all together, such as estimator, metric(or loss), hyperparameter ranges, we are going to run Hyperparameter Tuning Job.

In [17]:
max_jobs = 12
%store max_jobs

tuner = sagemaker.tuner.HyperparameterTuner(
    estimator,
    'TrainLoss',
    objective_type='Minimize',
    metric_definitions=metric_definitions,
    hyperparameter_ranges=hyperparameter_ranges,
    base_tuning_job_name='yolo-htj-batch-0',
    max_jobs=max_jobs,
    max_parallel_jobs=3
)

train_start_time = datetime.now()
%store train_start_time

tuner.fit(
    {
        "train": train_channel,
        "test": test_channel,
        "images": image_channel
    },
    include_cls_metadata=False
)

Stored 'max_jobs' (int)
Stored 'train_start_time' (datetime)


Once you call *fit()* method, you can check the progress on [AWS Console](https://console.aws.amazon.com).

<img src="Assets/TrainingJobStatus.png" />

and, of course, you can check progress out on the notebook using Sagemaker Python SDK

In [22]:
# wait for the first job is going to be ready to trating
time.sleep(10)

training_jobs = smclient.list_training_jobs(NameContains=tuner.base_tuning_job_name, StatusEquals='InProgress')
training_job_summaries = training_jobs['TrainingJobSummaries']
training_job_name = training_job_summaries[0]['TrainingJobName'].rsplit('-', 2)[0]
%store training_job_name

Stored 'training_job_name' (str)


In [23]:
analytics = sagemaker.HyperparameterTuningJobAnalytics(training_job_name)
analytics.dataframe()

,lr,momentum,optimizer,wd,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime
0,0.007768,0.940174,"""rmsprop""",0.820159,yolo-htj-batch-0-200320-1546-003-ef269a51,InProgress,None,None,None
1,0.000227,0.556847,"""adam""",0.912397,yolo-htj-batch-0-200320-1546-002-6c81679f,InProgress,None,None,None
2,0.000613,0.046775,"""sgd""",0.975574,yolo-htj-batch-0-200320-1546-001-440c6ec3,InProgress,None,None,None


Now, we are going to wait for all training job is completed, it will take few minues..

In [26]:
def fetch_n_completed_jobs(base_job_name, max_jobs, start_time):
    counts = 0
    date_suffix = date.today().strftime('%y%m%d')
    job_name = f'{base_job_name}-{date_suffix}'
    for status in ['Completed', 'Failed', 'Stopped']:
        counts += len(smclient.list_training_jobs(
            NameContains=job_name,
            StatusEquals='Completed',
            MaxResults=max_jobs,
            CreationTimeAfter=start_time,
        )['TrainingJobSummaries'])
    return counts
 

def wait_for_training(base_job_name, max_jobs, start_time):
    completed_jobs = 0
    while completed_jobs < max_jobs:
        print(f'[{base_job_name}] {completed_jobs}/{max_jobs} of training jobs are completed...')
        completed_jobs = fetch_n_completed_jobs(base_job_name, max_jobs, start_time)
        time.sleep(30)
    print(f'All({max_jobs}) training jobs are completed!!')

In [ ]:
wait_for_training(tuner.base_tuning_job_name, max_jobs, train_start_time)

[yolo-htj-batch-0] 0/12 of training jobs are completed...
[yolo-htj-batch-0] 0/12 of training jobs are completed...
